First download cat-mel_2bar_big.tar from [https://github.com/magenta/magenta/tree/main/magenta/models/music_vae](https://github.com/magenta/magenta/tree/main/magenta/models/music_vae)

In [ ]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from magenta.models.music_vae import configs
from magenta.models.music_vae import TrainedModel
import note_seq
# import numpy as np
# import tensorflow.compat.v1 as tf


print("Done with init")

checkpoint_dir_or_path = "./cat-mel_2bar_big.tar"
config = configs.CONFIG_MAP["cat-mel_2bar_big"]
num_outputs = 100
temperature = 0.5

model = TrainedModel(config, batch_size=num_outputs,checkpoint_dir_or_path=checkpoint_dir_or_path)
results = model.sample(
	n=num_outputs,
	length=config.hparams.max_seq_len,
	temperature=temperature)


encoded_results = []
skipped_results = set()
for i, result in enumerate(results):
    try:
        encoded_result = model.encode([result])
        encoded_results.append(encoded_result)
    except Exception:
        skipped_results.add(i)

print("skipped: ", skipped_results)

for i, ns in enumerate(results):
    if i not in skipped_results:
        note_seq.sequence_proto_to_midi_file(ns, os.path.join("./bruh", "output_%s.mid" % (str(i))))

Done with init
INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 100, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]



/home/eiffert/.local/lib/python3.8/site-packages/magenta/models/music_vae/lstm_utils.py:94: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  tf.layers.dense(
/home/eiffert/.local/lib/python3.8/site-packages/magenta/contrib/rnn.py:749: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._kernel = self.add_variable(
/home/eiffert/.local/lib/python3.8/site-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._bias = self.add_variable(


INFO:tensorflow:Unbundling checkpoint.


/home/eiffert/.local/lib/python3.8/site-packages/magenta/models/music_vae/base_model.py:195: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  mu = tf.layers.dense(
/home/eiffert/.local/lib/python3.8/site-packages/magenta/models/music_vae/base_model.py:200: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  sigma = tf.layers.dense(


INFO:tensorflow:Restoring parameters from /tmp/tmp0tfc50vu/cat-mel_2bar_big.ckpt
skipped:  {35, 6, 79}


In [ ]:
import os

dirname = "./bruh"
for filename in os.listdir(dirname):
    sequence = note_seq.midi_file_to_note_sequence(os.path.join(dirname, filename))
    print(model.encode([sequence]))
    note_seq.plot_sequence(sequence)

In [20]:
import os
import numpy as np

mins = []
maxs = []

dirname = "./bruh"
for i, filename in enumerate(os.listdir(dirname)):
    sequence = note_seq.midi_file_to_note_sequence(os.path.join(dirname, filename))
    _, encoding, _ = model.encode([sequence])
    if i == 0:
        mins = encoding[0]
        maxs = encoding[0]
    else:
        mins = np.minimum(mins, encoding[0])
        maxs = np.maximum(maxs, encoding[0])

print(mins)
print(maxs)
print(maxs-mins)

[-0.15566775 -0.06380363 -0.1427754  -0.12098837 -0.23931804 -0.19090691
 -0.30640543 -0.11970383 -0.31916255 -0.12051144 -0.10217745 -0.3838974
 -2.5845594  -0.25646472 -0.90147936 -0.15994017 -0.16984135 -0.2553321
 -0.15236725 -0.238052   -0.48421723 -0.2948621  -0.12799463 -0.17665614
 -1.9917369  -0.11317943 -0.10051731 -0.3287616  -0.18214521 -0.36535525
 -0.13093847 -0.19044437 -0.07406708 -0.16629715 -0.11169547 -0.2572921
 -1.8133262  -0.10236037 -0.08028846 -0.19275817 -0.13771465 -2.776007
 -0.4035515  -0.10508877 -0.0788838  -0.1319366  -0.24980894 -0.18008631
 -0.17982222 -0.12747473 -0.27703488 -0.13317007 -0.10235586 -0.21979803
 -0.27355206 -0.06850907 -0.09801304 -0.37491578 -0.25400242 -0.14695685
 -0.12669393 -0.313899   -2.0086267  -0.33200783 -0.08909269 -0.11247777
 -0.40140334 -0.27848172 -0.08494796 -0.09876963 -0.31852573 -0.11849488
 -0.11405692 -2.461828   -0.35675162 -0.11330548 -0.1863347  -0.26061505
 -0.24644354 -0.1698134  -0.1437873  -0.17845412 -0.1098

In [1]:
import os
os.environ['TF_CPP_MIN_LOG_LEVEL'] = '3'
from magenta.models.music_vae import configs
from magenta.models.music_vae import TrainedModel
import note_seq
import numpy as np
# import tensorflow.compat.v1 as tf
from IPython.display import clear_output
import time

print("Done with init")

checkpoint_dir_or_path = "./cat-mel_2bar_big.tar"
config = configs.CONFIG_MAP["cat-mel_2bar_big"]
temperature = 0.5

model = TrainedModel(config, batch_size=2,checkpoint_dir_or_path=checkpoint_dir_or_path)

# z = np.array([
#     _slerp(mu[0], mu[1], t) for t in np.linspace(0, 1, FLAGS.num_outputs)])
# results = model.decode(
#     length=config.hparams.max_seq_len,
#     z=z,
#     temperature=FLAGS.temperature)



/home/eiffert/.local/lib/python3.8/site-packages/tqdm/auto.py:21: TqdmWarning: IProgress not found. Please update jupyter and ipywidgets. See https://ipywidgets.readthedocs.io/en/stable/user_install.html
  from .autonotebook import tqdm as notebook_tqdm


Done with init
INFO:tensorflow:Building MusicVAE model with BidirectionalLstmEncoder, CategoricalLstmDecoder, and hparams:
{'max_seq_len': 32, 'z_size': 512, 'free_bits': 0, 'max_beta': 0.5, 'beta_rate': 0.99999, 'batch_size': 2, 'grad_clip': 1.0, 'clip_mode': 'global_norm', 'grad_norm_clip_to_zero': 10000, 'learning_rate': 0.001, 'decay_rate': 0.9999, 'min_learning_rate': 1e-05, 'conditional': True, 'dec_rnn_size': [2048, 2048, 2048], 'enc_rnn_size': [2048], 'dropout_keep_prob': 1.0, 'sampling_schedule': 'inverse_sigmoid', 'sampling_rate': 1000, 'use_cudnn': False, 'residual_encoder': False, 'residual_decoder': False, 'control_preprocessing_rnn_size': [256]}
INFO:tensorflow:
Encoder Cells (bidirectional):
  units: [2048]

INFO:tensorflow:
Decoder Cells:
  units: [2048, 2048, 2048]

Instructions for updating:
Use `tf.cast` instead.


/home/eiffert/.local/lib/python3.8/site-packages/magenta/models/music_vae/lstm_utils.py:94: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  tf.layers.dense(
/home/eiffert/.local/lib/python3.8/site-packages/magenta/contrib/rnn.py:749: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._kernel = self.add_variable(
/home/eiffert/.local/lib/python3.8/site-packages/magenta/contrib/rnn.py:751: UserWarning: `layer.add_variable` is deprecated and will be removed in a future version. Please use the `layer.add_weight()` method instead.
  self._bias = self.add_variable(


Instructions for updating:
Please use `keras.layers.Bidirectional(keras.layers.RNN(cell))`, which is equivalent to this API
Instructions for updating:
Please use `keras.layers.RNN(cell)`, which is equivalent to this API
Instructions for updating:
`scale_identity_multiplier` is deprecated; please combine it into `scale_diag` directly instead.
INFO:tensorflow:Unbundling checkpoint.


/home/eiffert/.local/lib/python3.8/site-packages/magenta/models/music_vae/base_model.py:195: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  mu = tf.layers.dense(
/home/eiffert/.local/lib/python3.8/site-packages/magenta/models/music_vae/base_model.py:200: UserWarning: `tf.layers.dense` is deprecated and will be removed in a future version. Please use `tf.keras.layers.Dense` instead.
  sigma = tf.layers.dense(


INFO:tensorflow:Restoring parameters from /tmp/tmpdi4nagkn/cat-mel_2bar_big.ckpt


In [2]:
result = []
encoded_result = []
while True:
    result = model.sample(
        n=1,
        length=config.hparams.max_seq_len,
        temperature=temperature)[0]
    try:
        _, encoded_result, _ = model.encode([result])
        encoded_result = encoded_result[0]
        break
    except Exception:
        pass


step_norm = config.hparams.z_size * 0.3
liked_seq = result
while True:
    clear_output(wait=True)

    print("step norm:", step_norm)

    _, original, _ = model.encode([result])
    original = original[0]
    
    dv = np.random.triangular(-1, 0, 1, size=np.shape(original))
    dv = dv / np.linalg.norm(dv)
    dv = step_norm * dv
    different = original + dv

    different_seq = model.decode(
        length=config.hparams.max_seq_len,
        z=np.array([different]),
        temperature=temperature)[0]

    note_seq.plot_sequence(liked_seq)
    note_seq.play_sequence(liked_seq)
    time.sleep(0.2)
    note_seq.plot_sequence(different_seq)
    note_seq.play_sequence(different_seq)

    done = False
    while True:
        choice = input("? ")
        if choice == "1":
            step_norm = step_norm * 0.9
            break
        elif choice == "2":
            liked_seq = different_seq
            break
        elif choice == "3" or choice == "":
            done = True
            break
    if done:
        break

note_seq.sequence_proto_to_midi_file(liked_seq, os.path.join("./liked", "output_%s.mid" % time.strftime('%Y-%m-%d_%H%M%S')))

step norm: 124.41600000000001


Loading BokehJS ...

Loading BokehJS ...